# SparkML Logistic Regression Model
**Goal:** Build a logistic regression model using the taxicab data set to predict payment type.


### Create a Spark Session

In [ ]:
from pyspark.sql import SparkSession

# create a spark session
spark = SparkSession \
    .builder \
    .appName("spark_job") \
    .getOrCreate()

sc = spark.sparkContext

### Load the input data file


In [ ]:
### Load Data
input_path='<path-to>/green_tripdata_2024-01.parquet'
raw_df = spark \
        .read \
        .parquet(input_path)

raw_df.show()


### Clean and filter data

In [ ]:
# filter for payment types 1 and 2 only
input_df = raw_df.filter('...')

### Define Functions to vectorize input data

In [ ]:
from pyspark.ml import Pipeline, Estimator
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.sql import DataFrame as SparkDataFrame

# Prepare data for machine learning
# from dataframe categorical and numeric columns create label and features

def vectorizeCategories(labelCol: str, categoricalColumns:list[str]) -> list[Estimator]:
  stages = [] # stages in Pipeline
  for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer:Estimator = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder:Estimator = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add categorical stagess
    stages += [stringIndexer, encoder]
  #add label category
  label_stringIdx = StringIndexer(inputCol=labelCol, outputCol="label")
  stages += [label_stringIdx] 
  return stages

def createVectorizePipeline(labelCol: str, categoricalCols:list[str], numericCols:list[str]) -> Pipeline:
  categoricalStages = vectorizeCategories(labelCol, categoricalCols)
  assemblerInputs = [c + "classVec" for c in categoricalCols] + numericCols
  assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
  allStages:list[Estimator | VectorAssembler] = categoricalStages + [assembler]
  partialPipeline = Pipeline().setStages(allStages) # type: ignore
  return partialPipeline


### Select category to predict and input features

In [ ]:
# label column is the feature to predict
label_col = '<feature to predict>'
categorical_feature_cols = [<categorical columns>]
numeric_feature_cols=[<numeric columns>]
vectorizePipeline = createVectorizePipeline(label_col, categorical_feature_cols, numeric_feature_cols)

# create vector dataframe
vectorizedModel = vectorizePipeline.fit(input_df)
vectorized_df = vectorizedModel.transform(input_df)
selectedcols = ["label", "features"] + [label_col] + categorical_feature_cols + numeric_feature_cols
mldata_df = vectorized_df.select(selectedcols)


### Train model

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(
    featuresCol="features", labelCol="label")

# split the data
train_df, test_df = mldata_df.randomSplit([0.8, 0.2])
# select the features and label
train_df = train_df.select("features", "label")
test_df = test_df.select("features", "label")


lrModel = lr.fit(train_df)
prediction_df = lrModel.transform(test_df)
prediction_df.show()

### Evaluate model

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(
    labelCol="label")
evaluator.evaluate(prediction_df)